In [1]:
import pandas as pd
import numpy as np

df=pd.read_csv('lab_1_data.csv')
df

,Unnamed: 0,x1,x2,u,e,y
0,0,-1.0,-1.0,-2.5700,0.184213,-2.385787
1,1,-1.0,-0.5,-1.0200,0.224978,-0.795022
2,2,-1.0,0.0,-0.4700,0.476234,0.006234
3,3,-1.0,0.5,-0.9200,0.066598,-0.853402
4,4,-1.0,1.0,-2.3700,-0.198972,-2.568972
5,5,-0.5,-1.0,-1.3425,-0.052922,-1.395422
6,6,-0.5,-0.5,0.2075,0.370028,0.577528
7,7,-0.5,0.0,0.7575,0.133969,0.891469
8,8,-0.5,0.5,0.3075,0.349004,0.656504
9,9,-0.5,1.0,-1.1425,-0.762287,-1.904787


In [2]:
#предлагаю если что оставшийся импорт библиотек сделать здесь
import scipy.stats

In [3]:
#наша новая функция(с доп регрессором)
def u_func(x1, x2):
    return 2 + 2.5*x1 + 0.03*x1*x1 + 0.1*x2 - 2*x2*x2+0.001*x2**3

In [4]:
#наши старые х1 и х2
x1 = np.array([-1, -0.5, 0.5, 1])
x2 = np.array([-1, -0.5, 0, 0.5, 1])
u = np.array([])

for i in x1:
    for j in x2:
        u = np.append(u, u_func(i, j))

In [5]:
#изменила генерацию e, теперь в функцию подается среднеквадратичное отклонение
n = len(u)
p = 0.1 # доля
u_average = np.full(n, np.mean(u))

# считаем ошибку наблюдения
w_sq = np.dot((u - u_average).transpose(), (u - u_average)) / (n - 1)
d = (p * w_sq)**(1/2)
e = np.random.normal(0, d, n)

In [6]:
# дополнительные столбцы в датафрейме
df['x1^2'] = df['x1']**2
df['x2^2'] = df['x2']**2
df['x2^3'] = df['x2']**3
df

,Unnamed: 0,x1,x2,u,e,y,x1^2,x2^2,x2^3
0,0,-1.0,-1.0,-2.5700,0.184213,-2.385787,1.00,1.00,-1.000
1,1,-1.0,-0.5,-1.0200,0.224978,-0.795022,1.00,0.25,-0.125
2,2,-1.0,0.0,-0.4700,0.476234,0.006234,1.00,0.00,0.000
3,3,-1.0,0.5,-0.9200,0.066598,-0.853402,1.00,0.25,0.125
4,4,-1.0,1.0,-2.3700,-0.198972,-2.568972,1.00,1.00,1.000
5,5,-0.5,-1.0,-1.3425,-0.052922,-1.395422,0.25,1.00,-1.000
6,6,-0.5,-0.5,0.2075,0.370028,0.577528,0.25,0.25,-0.125
7,7,-0.5,0.0,0.7575,0.133969,0.891469,0.25,0.00,0.000
8,8,-0.5,0.5,0.3075,0.349004,0.656504,0.25,0.25,0.125
9,9,-0.5,1.0,-1.1425,-0.762287,-1.904787,0.25,1.00,1.000


In [7]:
class MyLinearRegression:
    
    def __init__(self):
        self.coef_ = None
        self.intercept_ = None
        self.k = None
        self.for_sigma = None
        
        
    def fit(self, X, y):
        X = np.array(X)
        y = np.array(y)

        X = np.hstack((np.ones((X.shape[0], 1)), X))

        self.k = np.linalg.inv(X.T @ X) @ X.T @ y
        self.for_sigma = np.linalg.inv(X.T @ X)

        self.coef_ = self.k[::][1:]
        self.intercept_ = self.k[::][0]
        
    def predict(self, X):
        X = np.hstack((np.ones((X.shape[0], 1)), X))
        y_pred = X @ self.k        
        return y_pred
    
    def get_param(self):
        return self.coef_, self.intercept_

In [8]:
LinModel = MyLinearRegression()

In [9]:
#добавила в х новый регрессор
x = df[['x1','x1^2','x2', 'x2^2','x2^3']]
y = df['y']

In [10]:
LinModel.fit(x,y)
y_predict = LinModel.predict(x)

In [11]:
LinModel.get_param()

(array([ 2.57261407,  0.53351184, -0.1873366 , -2.29696999,  0.07873997]),
 1.9342890165915396)

In [12]:
tetta_wave_j = LinModel.get_param()[1] 
tetta = LinModel.get_param()[0] 
tetta = np.append(tetta_wave_j, tetta) 
print(tetta)

[ 1.93428902  2.57261407  0.53351184 -0.1873366  -2.29696999  0.07873997]


### Для вычисления сигмочек и тд

In [13]:

sigma = np.sqrt(p*w_sq)
e_predict = y - y_predict
sigma_sq_predict = e_predict.T @ e_predict / (n - 6)
print(sigma_sq_predict)
F = sigma_sq_predict/sigma**2
FT = scipy.stats.f.ppf(q=1-0.05, dfn=9999, dfd=16)
print('F =', F)
print('FT =', FT)
print('F < FT:', F < FT)

0.07804841451360674
F = 0.1607863065777916
FT = 2.0102414060038662
F < FT: True


## Второй пункт. Доверительные интервалы

In [14]:
true_tetta = [2,2.5,0.03,0.1,-2,0.001]
for j in range(0,6):
    sigma_tetta_wave_j = (LinModel.for_sigma[j][j]*sigma_sq_predict)**(1/2)
    print(f'Минимальное значение:{tetta[j]-FT*sigma_tetta_wave_j}<={true_tetta[j]}<= Макс.знач{tetta[j]+FT*sigma_tetta_wave_j}')

Минимальное значение:1.6477512272081811<=2<= Макс.знач2.220826805974898
Минимальное значение:2.4137684493255573<=2.5<= Макс.знач2.7314596937880404
Минимальное значение:0.1986358674434282<=0.03<= Макс.знач0.8683878175732967
Минимальное значение:-0.7209425813343396<=0.1<= Макс.знач0.3462693758466048
Минимальное значение:-2.5971599993061782<=-2<= Макс.знач-1.9967799803416377
Минимальное значение:-0.513242712001087<=0.001<= Макс.знач0.6707226521232745


## Третий пункт. Гипотеза о незначимости параметра

In [15]:
#значение F критического, с которым будем сравнивать
F_crit=scipy.stats.f.isf(0.05,1,n-6)
print(F_crit)

4.600109936669421


In [18]:
for j in range(0,6):
    djj=LinModel.for_sigma[j][j]*sigma_sq_predict
    F = (tetta[j]**2)/djj
    print(f" Номер: {j},Значение F={F}, F<fcrit  {F<F_crit}")

 Номер: 0,Значение F=184.1515369177979, F<fcrit  False
 Номер: 1,Значение F=1059.9739921715877, F<fcrit  False
 Номер: 2,Значение F=10.256910689804146, F<fcrit  False
 Номер: 3,Значение F=0.4980814126985353, F<fcrit  True
 Номер: 4,Значение F=236.5999243496663, F<fcrit  False
 Номер: 5,Значение F=0.0714938878832311, F<fcrit  True
